In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import pathlib
python_path = pathlib.Path('.').absolute().parent/'python'
os.sys.path.insert(1, str(python_path))

In [3]:
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from tqdm import trange
import wandb

In [4]:
from vocam.diff_pin_costs import DiffFrameTranslationCost, DiffFrameVelocityCost
from vocam.qpnet import DataUtils, QPNet
import vocam.qpnet

In [5]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")

viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


In [6]:
run = wandb.init(project="vocam", group="kuka_qpnet_data", entity="hjzhu")

wandb: Currently logged in as: hjzhu. Use `wandb login --relogin` to force relogin


In [7]:
# robot configuration
wandb.config.nq = model.nq
wandb.config.nv = model.nv
wandb.config.n_col = 5
wandb.config.n_vars = 3 * model.nq * wandb.config.n_col + 2 * model.nq
wandb.config.u_max = [2.5, 2.5, 2.5, 1.5, 1.5, 1.5, 1.0]
wandb.config.dt = 0.05
wandb.config.q_init = np.array([0.0, 0.3, 0.0, -0.8, -0.6,  0.0, 0.0])
wandb.config.x_init = np.concatenate([wandb.config.q_init, pin.utils.zero(model.nv)])
wandb.config.dq_noise = 0.8
wandb.config.default_goal = [0.6, 0.4, 0.7]

# ioc optimization parameters
wandb.config.isvec = True
wandb.config.lr_qp = 1e-1
wandb.config.max_it = 100
wandb.config.task_horizon = 30
wandb.config.loss_threshold = 0.05
wandb.config.convergence_threshold = 5e-4
wandb.config.distance_threshold = 0.02

# target sample range
wandb.config.lb = [ 0.4, 0.4, 0.0]
wandb.config.ub = [ 1.0, 1.0, 1.2]
wandb.config.diff_range = 2.0
wandb.config.dist_ub = 0.8
wandb.config.dist_lb = 0.4

In [8]:
def task_loss(robot, q_pred, x_des, nq, n_col):
    dtc = DiffFrameTranslationCost.apply
    dvc = DiffFrameVelocityCost.apply
    model, data = robot.model, robot.data
    f_id = model.getFrameId("EE")

    loss = 3.5e1*torch.linalg.norm(dtc(q_pred[-2*nq:], model, data, f_id) - x_des)
    loss += 1.5e0*torch.linalg.norm(dvc(q_pred[-2*nq:], torch.zeros(nq), model, data, f_id)) # asking for zero velocity
    loss += 2e-1*torch.linalg.norm(q_pred[-2*nq:-nq]) # joint regularization

    for i in range(n_col):    
        loss += 1e0*torch.linalg.norm(dtc(q_pred[(3*i)*nq: (3*i+2)*nq], model, data, f_id) - x_des)
        loss += 5e-1*torch.linalg.norm(dvc(q_pred[(3*i)*nq: (3*i+2)*nq], q_pred[(3*i+2)*nq:(3*i+3)*nq], model, data, f_id)) # asking for zero velocity
        loss += 1e-2*torch.linalg.norm(q_pred[(3*i+2)*nq: (3*i+3)*nq]) # control regularization
        loss += 2e-1*torch.linalg.norm(q_pred[(3*i+1)*nq: (3*i+2)*nq]) # velocity regularization
        loss += 3e-3*torch.linalg.norm(q_pred[(3*i)*nq: (3*i+1)*nq]) # joint regularization
    
    return loss

In [9]:
du = DataUtils(robot, task_loss, wandb.config, viz)
n_tasks = 100
du.generate(n_tasks)
n_run = run.name.split('-')[-1]
data_dir = '../data/'
file_path = data_dir + 'data_' + str(n_tasks) + '_' + n_run + '.pt'
du.save(file_path)

# artifact
data_artifacts = wandb.Artifact('qpnet_datasets', type='dataset')
data_artifacts.add_dir(data_dir)
run.log_artifact(data_artifacts)

 70%|███████   | 7/10 [01:30<00:36, 12.14s/it]

0.035281332517690556


100%|██████████| 10/10 [02:11<00:00, 13.15s/it]
wandb: Adding directory to artifact (./../data)... Done. 0.1s
